In [1]:
import numpy as np
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import pvlib
sys.path.append(os.getcwd())
from persistence import PVForecast


In [7]:
def get_s_persistance(ghi, horizon, freq='10s', location=None):
    """
    get a single persistance forecast
    """

    if location is None:
        location = pvlib.location.Location(latitude=59.973218,
                                       longitude=11.051604,
                                       altitude=130,
                                       tz='CET')

    persistance = PVForecast(measured_signal=ghi, 
                             binning=None,
                             f_h=horizon,
                             s_h = '0s',
                             f_i=freq,
                             location=location)
                             
    sp = persistance.smart_persistance()     

    return sp

def get_clearsky2(location, index, G_0=1360, max_sza=90, kind='clear') -> pd.Series:
    """
    Calculate clearness.
    
    :param G_0: float
        the extra terrestrial radiation received by the Earth from the Sun
    :return: G_clear : pandas.Series
    
    """
    
    # Get solar zenith angle (SZA):
    sza = location.get_solarposition(index)['zenith']
    
    

    # Get clearsky
    if kind=='clearsky':
        G_clear = location.get_clearsky(sza.index)
    elif kind == 'clear':
        # Calculate cos(SZA):
        cos_zenith = pvlib.tools.cosd(sza)
    else:
        print('select clear or clearsky, default to clear')
        # Calculate cos(SZA):
        cos_zenith = pvlib.tools.cosd(sza)
    
    # Calculate clearness:
    G_clear = G_0 * np.maximum(cos_zenith, np.cos(np.radians(max_sza)))
    
    return G_clear


def get_clearsky(ghi, horizon, freq='10s', location=None):
    """
    get a single persistance forecast
    """

    if location is None:
        location = pvlib.location.Location(latitude=59.973218,
                                       longitude=11.051604,
                                       altitude=130,
                                       tz='CET')

    persistance = PVForecast(measured_signal=ghi, 
                             binning=None,
                             f_h=horizon,
                             s_h = '0s',
                             f_i=freq,
                             location=location)
                             
    GHI_clear = persistance.get_clearness()     

    return GHI_clear

    
    
def get_spersistance_df(index, horizon_steps=90, location=None, freq='10s', ghi_data_path=None, ghi_df=None,revnorm=False):
    """
    get datafram of persistance forecasts, where the index indicates the time at which the persistance forecast is issued
    With a given frequency
    A given amount of steps forward in time
    """
    if ghi_data_path == None:
        ghi_data_path = 'Data_GHI/KZPR_IRHoSunP'
        
    # dates = np.sort(list(set(pd.to_datetime(index).date)))
    try:
        dates = np.sort(list(set(index.dt.date)))
    except:
        dates = np.sort(list(set(index.date)))
    #columns = [(i*10)+10 for i in range(horizon_steps)]
    sp_df = pd.DataFrame()

    for day in dates:
        print(day)
    
        day_ind = "".join(map(lambda x: x, str(day).split("-")))

        if ghi_df is None:    
            if (ghi_data_path.split('/')[-1].split('_')[0] == 'processed'):
                ghi = (pd.read_pickle(f'{ghi_data_path}/{day_ind}.pkl').tz_localize('CET')+1)*(1367/2)
            else:
                ghi = pd.read_pickle(f'{ghi_data_path}/{day_ind}/{day_ind}.pkl').set_index('date_time').tz_localize('CET')
        else:
            ghi = ghi_df[ghi_df.index.date == day]
            
            if revnorm==True:
                ghi = (ghi + 1)*(1367/2)
    
        if location is None:
            location = pvlib.location.Location(latitude=59.973218,
                                           longitude=11.051604,
                                           altitude=130,
                                           tz='CET')
        elif location == 'sunpoint_target_11':
            location = pvlib.location.Location(latitude=59.6625, longitude=5.9538,
                                           altitude=8,
                                           tz='CET')
        else:
            raise NotImplementedError
        # first horizon
        sp = get_s_persistance(ghi=ghi, horizon=freq, freq=freq, location=location)

        # rest of horizons
        for step in range(2,horizon_steps+1):
            if freq == '10s':
                h_seconds = step*10
                sp_step = get_s_persistance(horizon=f'{h_seconds}s', freq=freq, ghi=ghi, location=location)
            elif freq == '1h':
                h_seconds = step
                sp_step = get_s_persistance(horizon=f'{h_seconds}h', freq=freq, ghi=ghi, location=location)
            
        
            sp = pd.concat([sp,sp_step],axis=1)

        # Shift forecast to correspond to issued timestamp
        sp.columns = [i*10 for i in range(1,horizon_steps+1)]
        for i,col_name in enumerate(sp.columns):
            try:
                sp.iloc[:-i-1,i] = sp.iloc[i+1:,i]
            except:
                raise ValueError('Not enough data to shift forecast')
        
        # concat each day
        sp_day = sp.loc[index[index.dt.date == day],:]
        sp_df = pd.concat([sp_df, sp_day], axis=0)
    return sp_df


In [35]:
# file_path_ghi = "C:\\Users\\nikla\\Documents\\phd\\paper2\\code\\IFE_dataset_model\\irr_df_20210622_20240618.pkl"
# irr_df = pd.read_pickle(file_path_ghi)

path = 'C:\\Users\\nikla\\Documents\\phd\\paper2\\code\\IFE_dataset_model\\trainval_df_preprocessed.pkl'
trainval_df = pd.read_pickle(path)
ghi = trainval_df['GHI'] # notably this does not include a test set

index = pd.read_pickle('C:\\Users\\nikla\\Documents\\phd\\paper2\\code\\IFE_dataset_model\\trainval_C_index.pkl')

index_filtered = index.iloc[::3]
ghi_rolling = ghi.rolling(window=3).mean()#[index_filtered]


ghi_rolling


In [39]:
ghi

2022-06-09 05:52:10+02:00    40.999981
2022-06-09 05:52:20+02:00    40.999981
2022-06-09 05:52:30+02:00    40.999981
2022-06-09 05:52:40+02:00    40.999981
2022-06-09 05:52:50+02:00    40.999981
                               ...    
2024-02-29 16:05:10+01:00     7.999988
2024-02-29 16:05:20+01:00     7.999988
2024-02-29 16:05:30+01:00     7.999988
2024-02-29 16:05:40+01:00     7.999988
2024-02-29 16:05:50+01:00     7.999988
Name: GHI, Length: 111388, dtype: float32

In [ ]:


pers = get_spersistance_df(index_filtered, horizon_steps=30, location=None, freq='30s', ghi_data_path=None, ghi_df=ghi_rolling)



In [ ]:
pers.to_pickle('persistence_30s.pkl')

In [13]:
import os
print("Current working dir:", os.getcwd())

# List what's inside the res directory
import os
parent = os.path.dirname(os.getcwd())
res = os.path.join(parent, 'res')
print("res dir path:", res)
print("Files in res dir:", os.listdir(res))

Current working dir: c:\Users\nikla\Documents\phd\paper2\code\models
res dir path: c:\Users\nikla\Documents\phd\paper2\code\res
Files in res dir: ['data.py', 'ife_data.py', 'model.py', 'readyDat_time_csi_nights_allseason_nora_cam.npz', '__init__.py', '__pycache__']


In [3]:
import sys
import os

# Get the current working directory (e.g., the directory of the notebook)
current_dir = os.getcwd()

# Go up to project root
parent_dir = os.path.dirname(current_dir)

# Add the 'res' directory to sys.path
res_path = os.path.join(parent_dir, 'res')
sys.path.append(res_path)
from data import data_import
sunpoint_target = [59.6625, 5.9538]
start_date = "2016-01-01 00:30:00"
end_date = "2020-12-31 23:30:00"    
sun_index = pd.date_range(start=start_date, end = end_date, freq = '1h', tz='CET')
train,train_target,valid,valid_target,_,test_target = data_import(parent_dir)
# sun_ghi_data = 

sun_ghi = pd.DataFrame(index=sun_index, data=np.concatenate([test_target,valid_target,train_target]), columns=['GHI'])


In [5]:
sun_ghi.index

DatetimeIndex(['2016-01-01 00:30:00+01:00', '2016-01-01 01:30:00+01:00',
               '2016-01-01 02:30:00+01:00', '2016-01-01 03:30:00+01:00',
               '2016-01-01 04:30:00+01:00', '2016-01-01 05:30:00+01:00',
               '2016-01-01 06:30:00+01:00', '2016-01-01 07:30:00+01:00',
               '2016-01-01 08:30:00+01:00', '2016-01-01 09:30:00+01:00',
               ...
               '2020-12-31 14:30:00+01:00', '2020-12-31 15:30:00+01:00',
               '2020-12-31 16:30:00+01:00', '2020-12-31 17:30:00+01:00',
               '2020-12-31 18:30:00+01:00', '2020-12-31 19:30:00+01:00',
               '2020-12-31 20:30:00+01:00', '2020-12-31 21:30:00+01:00',
               '2020-12-31 22:30:00+01:00', '2020-12-31 23:30:00+01:00'],
              dtype='datetime64[ns, CET]', length=43848, freq='h')

In [9]:
pers = get_spersistance_df(sun_ghi.index, horizon_steps=36, location="sunpoint_target_11", freq='1h', ghi_data_path=None, ghi_df=sun_ghi["GHI"])

2016-01-01


ValueError: Not enough data to shift forecast